In [ ]:
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from minerva import select, normalize
from minerva.iterable_dataset import MyDataset, MyIterableDataset

In [ ]:
torch.set_float32_matmul_precision("medium")

In [ ]:
n = 50000
dx = 10
num_relevant = 2
dy = 1
train_size = int(.66 * n)
val_size = int(.15 * n)
test_size = n - train_size - val_size

batch_size = 3000

In [ ]:
x = np.random.uniform(size=(n, dx))
expected = np.random.choice(dx, replace=False, size=num_relevant)
y = (
    np.random.uniform(size=(1, dy, num_relevant)) @ np.expand_dims(x[:, expected], axis=2)
)[:, :, 0]

In [ ]:
features_cols = [f'f{n}' for n in range(dx)]
float_features = features_cols
cat_features = []
target_cols = [f'y{n}' for n in range(dy)]
target_names = target_cols
xdf = pd.DataFrame(
        x,
        columns=features_cols
    )                                                                                                
ydf = pd.DataFrame(                                                                                
    y,
    columns=target_cols                                                                         
)
data = pd.concat((xdf, ydf), axis=1) 
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size: train_size + val_size]
test_data = data.iloc[train_size + val_size:]

In [ ]:
dn = normalize.DatasetNormalizer(float_cols=features_cols + target_cols, categorical_cols=[])
train_data = dn.fit_transform(train_data)
val_data = dn.transform(val_data)
test_data = dn.transform(test_data)

In [ ]:
train_dataset = MyDataset(
    train_data,
    float_features,
    cat_features,
    target_names
)
val_dataset = MyDataset(
    val_data,
    float_features,
    cat_features,
    target_names
)
test_dataset = MyDataset(
    test_data,
    float_features,
    cat_features,
    target_names
)

In [ ]:
train_dataloader = MyIterableDataset(train_dataset, batch_size=batch_size)
val_dataloader = MyIterableDataset(val_dataset, batch_size=batch_size)
test_dataloader = MyIterableDataset(test_dataset, batch_size=batch_size)

In [ ]:
dimension_of_residual_block = 200
lr = 1e-4
num_res_layers = 3
regularization_coef = 1e1
max_epochs = 500

In [ ]:
selector = select.Selector(
    feature_cols=features_cols,
    cat_feature_encoder=dn.cat_enc,
    target_cols=target_names,
    dim1_max=dimension_of_residual_block,
    lr=lr,
    num_res_layers=num_res_layers,
    regularization_coef=regularization_coef,
)

In [ ]:
selector.set_loaders(train_dataloader, val_dataloader, test_dataloader)

In [ ]:
logger = TensorBoardLogger("tb_logs", name='linear')
trainer = pl.Trainer(
    gradient_clip_val=.5,
    accelerator="auto",
    log_every_n_steps=10,
    max_epochs=max_epochs,
    logger=logger
)

In [ ]:
trainer.fit(
    selector,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

In [ ]:
# print results
print(
    f'Normalised coefficients of the projection matrix:\n{selector.normalized_proj()}\n')
print(f'Selected features:\n{selector.selected_feature_names()}\n')
print(f'Expected features\n{expected}\n')
